In [142]:
import pandas as pd
import numpy as np
import json
import statsapi
import tqdm
import statistics

In [22]:
sched = statsapi.schedule(start_date='2020-03-23', end_date='2020-12-31',)
print(f"num games: {len(sched)}")

num games: 2431


In [31]:
sched_df = pd.DataFrame.from_records(sched)
sched_df.head()

,game_id,game_datetime,game_date,game_type,status,away_name,home_name,away_id,home_id,doubleheader,game_num,home_probable_pitcher,away_probable_pitcher,home_pitcher_note,away_pitcher_note,away_score,home_score,current_inning,inning_state,summary
0,605422,2020-03-26T17:10:00Z,2020-03-26,R,Scheduled,Washington Nationals,New York Mets,120,121,N,1,,,,,0,0,,,2020-03-26 - Washington Nationals @ New York M...
1,604457,2020-03-26T17:10:00Z,2020-03-26,R,Scheduled,Detroit Tigers,Cleveland Indians,116,114,N,1,"Bieber, Shane",,,,0,0,,,2020-03-26 - Detroit Tigers @ Cleveland Indian...
2,604265,2020-03-26T18:10:00Z,2020-03-26,R,Scheduled,Kansas City Royals,Chicago White Sox,118,145,N,1,,,,,0,0,,,2020-03-26 - Kansas City Royals @ Chicago Whit...
3,605229,2020-03-26T18:10:00Z,2020-03-26,R,Scheduled,Chicago Cubs,Milwaukee Brewers,112,158,N,1,,,,,0,0,,,2020-03-26 - Chicago Cubs @ Milwaukee Brewers ...
4,603974,2020-03-26T19:05:00Z,2020-03-26,R,Scheduled,New York Yankees,Baltimore Orioles,147,110,N,1,,,,,0,0,,,2020-03-26 - New York Yankees @ Baltimore Orio...


In [20]:
all_teams = set(g['home_name'] for g in sched if not 'all-stars' in g['home_name'].lower())

In [21]:
# Los Angeles Angels (85.5)
# Houston Astros (97)
# Oakland Athletics (89.5)
# Toronto Blue Jays (76)
# Atlanta Braves (92)
# Milwaukee Brewers (83.5)
# St. Louis Cardinals (88.5)
# Chicago Cubs (86)
# Arizona Diamondbacks (83)
# Los Angeles Dodgers (99)
# San Francisco Giants (71)
# Cleveland Indians (86)
# Seattle Mariners (67.5)
# Miami Marlins (63.5)
# New York Mets (86.5) 
# Washington Nationals (89.5)
# Baltimore Orioles (55.5)
# San Diego Padres (83)
# Philadelphia Phillies (85.5)
# Pittsburgh Pirates (70.5)
# Texas Rangers (78.5)
# Tampa Bay Rays (90)
# Boston Red Sox (89)
# Cincinnati Reds (83.5)
# Colorado Rockies (74)
# Detroit Tigers (55.5)
# Minnesota Twins (90.5)
# Chicago White Sox (84.5)
# New York Yankees (101)

30

In [38]:
# vegas o/u win totals
team_strengths = {
    'Los Angeles Angels': 85.5,
    'Houston Astros': 97,
    'Oakland Athletics': 89.5,
    'Toronto Blue Jays': 76,
    'Atlanta Braves': 92,
    'Milwaukee Brewers': 83.5,
    'St. Louis Cardinals': 88.5,
    'Chicago Cubs': 86,
    'Arizona Diamondbacks': 83,
    'Los Angeles Dodgers': 99,
    'San Francisco Giants': 71,
    'Cleveland Indians': 86,
    'Seattle Mariners': 67.5,
    'Miami Marlins': 63.5,
    'New York Mets': 86.5, 
    'Washington Nationals': 89.5,
    'Baltimore Orioles': 55.5,
    'San Diego Padres': 83,
    'Philadelphia Phillies': 85.5,
    'Pittsburgh Pirates': 70.5,
    'Texas Rangers': 78.5,
    'Tampa Bay Rays': 90,
    'Boston Red Sox': 89,
    'Cincinnati Reds': 83.5,
    'Colorado Rockies': 74,
    'Detroit Tigers': 55.5,
    'Minnesota Twins': 90.5,
    'Chicago White Sox': 84.5,
    'New York Yankees': 101,
    'Kansas City Royals':64.5
}

In [39]:
[t for t in all_teams if not t in team_strength]

[]

In [44]:
season = sched_df.copy(deep=True)
season = season[season.game_type == 'R']

In [53]:
def home_win_probabilities(h, a):
    """
    Home win o/u / Total o/u wins for each team 
    """
    home_strength = team_strengths[h]
    away_strength = team_strengths[a]
    
    total_strength = home_strength + away_strength
    
    return home_strength / total_strength
    
def simulate_game(game):
    home_team = game['home_name']
    away_team = game['away_name']
    home_team_p = home_win_probabilities(home_team, away_team)
    
    home_win = np.random.rand() < home_team_p
    
    if home_win:
        return home_team
    return away_team




In [54]:
season

,game_id,game_datetime,game_date,game_type,status,away_name,home_name,away_id,home_id,doubleheader,...,home_probable_pitcher,away_probable_pitcher,home_pitcher_note,away_pitcher_note,away_score,home_score,current_inning,inning_state,summary,winner
0,605422,2020-03-26T17:10:00Z,2020-03-26,R,Scheduled,Washington Nationals,New York Mets,120,121,N,...,,,,,0,0,,,2020-03-26 - Washington Nationals @ New York M...,Washington Nationals
1,604457,2020-03-26T17:10:00Z,2020-03-26,R,Scheduled,Detroit Tigers,Cleveland Indians,116,114,N,...,"Bieber, Shane",,,,0,0,,,2020-03-26 - Detroit Tigers @ Cleveland Indian...,Cleveland Indians
2,604265,2020-03-26T18:10:00Z,2020-03-26,R,Scheduled,Kansas City Royals,Chicago White Sox,118,145,N,...,,,,,0,0,,,2020-03-26 - Kansas City Royals @ Chicago Whit...,Chicago White Sox
3,605229,2020-03-26T18:10:00Z,2020-03-26,R,Scheduled,Chicago Cubs,Milwaukee Brewers,112,158,N,...,,,,,0,0,,,2020-03-26 - Chicago Cubs @ Milwaukee Brewers ...,Milwaukee Brewers
4,603974,2020-03-26T19:05:00Z,2020-03-26,R,Scheduled,New York Yankees,Baltimore Orioles,147,110,N,...,,,,,0,0,,,2020-03-26 - New York Yankees @ Baltimore Orio...,New York Yankees
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2426,605212,2020-09-27T19:10:00Z,2020-09-27,R,Scheduled,New York Mets,Miami Marlins,121,146,N,...,,,,,0,0,,,2020-09-27 - New York Mets @ Miami Marlins (Sc...,Miami Marlins
2427,604537,2020-09-27T19:10:00Z,2020-09-27,R,Scheduled,Toronto Blue Jays,Cleveland Indians,141,114,N,...,,,,,0,0,,,2020-09-27 - Toronto Blue Jays @ Cleveland Ind...,Cleveland Indians
2428,604633,2020-09-27T19:10:00Z,2020-09-27,R,Scheduled,Arizona Diamondbacks,Colorado Rockies,109,115,N,...,,,,,0,0,,,2020-09-27 - Arizona Diamondbacks @ Colorado R...,Arizona Diamondbacks
2429,604249,2020-09-27T19:20:00Z,2020-09-27,R,Scheduled,St. Louis Cardinals,Chicago Cubs,138,112,N,...,,,,,0,0,,,2020-09-27 - St. Louis Cardinals @ Chicago Cub...,Chicago Cubs


In [82]:
standings_resp = statsapi.standings_data()

division_standings = {}
for div_num in standings_resp:
    division_standings[standings_resp[div_num]['div_name']] = pd.DataFrame.from_records(standings_resp[div_num]['teams'])
    division_standings[standings_resp[div_num]['div_name']]["division"] = standings_resp[div_num]['div_name']
    division_standings[standings_resp[div_num]['div_name']]["league"] = standings_resp[div_num]['div_name'].rsplit(" ", 1)[0]

In [83]:
mlb_standings = pd.concat([division_standings[div] for div in division_standings])
mlb_standings = mlb_standings[['name', 'division', 'league']]
mlb_standings.head()

,name,division,league
0,Houston Astros,American League West,American League
1,Los Angeles Angels,American League West,American League
2,Oakland Athletics,American League West,American League
3,Seattle Mariners,American League West,American League
4,Texas Rangers,American League West,American League


In [84]:
games_won = season.winner.value_counts()
mlb_standings["wins"] = mlb_standings.name.map(lambda x:games_won[x])
mlb_standings.head()

,name,division,league,wins
0,Houston Astros,American League West,American League,87
1,Los Angeles Angels,American League West,American League,79
2,Oakland Athletics,American League West,American League,84
3,Seattle Mariners,American League West,American League,72
4,Texas Rangers,American League West,American League,76


In [118]:
def find_playoff_teams(mlb_standings):
    div_winners = []
    
    for div in mlb_standings.division.unique():
        winner = mlb_standings[mlb_standings.division == div].sort_values('wins', ascending=False).iloc[0]['name']
        div_winners.append(winner)
    
    
    wild_card = []
    wild_card_standings = mlb_standings[~mlb_standings.name.isin(div_winners)]
    
    for league in mlb_standings.league.unique():
        league_wc = wild_card_standings[wild_card_standings.league == league].sort_values('wins', ascending=False)['name'][:2]
        wild_card.extend(league_wc)
        
    return {'div_winners':div_winners, 'wild_card':wild_card}
        

In [181]:
SEASONS_TO_SIMULATE = 1000

playoff_appearances = {t:0 for t in all_teams}
div_wins = {t:0 for t in all_teams}
wc_appearances = {t:0 for t in all_teams}
win_totals = {t:[] for t in all_teams}

for season in tqdm.tqdm(range(SEASONS_TO_SIMULATE)):
    
    season = sched_df.copy(deep=True)
    season_standings = mlb_standings.copy(deep=True)
    
    # SHORTEN SEASON
    #season = season[1200:]
    
    
    season = season[season.game_type == 'R']
    season["winner"] = season.apply(simulate_game, axis=1)
    
    games_won = season.winner.value_counts()
    for team in games_won.to_dict():
        win_totals[team].append(games_won.to_dict()[team])
    season_standings["wins"] = mlb_standings.name.map(lambda x:games_won[x])
    
    playoff_teams = find_playoff_teams(season_standings)
    
    
    for team in playoff_teams['div_winners']:
        playoff_appearances[team] += 1
        div_wins[team] += 1
        
    for team in playoff_teams['wild_card']:
        playoff_appearances[team] += 1
        wc_appearances[team] += 1

100%|██████████| 1000/1000 [02:17<00:00,  7.25it/s]


In [182]:
playoff_appearances

{'Texas Rangers': 172,
 'Los Angeles Angels': 309,
 'Boston Red Sox': 475,
 'Philadelphia Phillies': 364,
 'New York Mets': 345,
 'Kansas City Royals': 14,
 'Minnesota Twins': 583,
 'Pittsburgh Pirates': 60,
 'Milwaukee Brewers': 335,
 'Cincinnati Reds': 325,
 'Miami Marlins': 7,
 'St. Louis Cardinals': 490,
 'Seattle Mariners': 25,
 'Toronto Blue Jays': 115,
 'Washington Nationals': 506,
 'Detroit Tigers': 2,
 'Arizona Diamondbacks': 325,
 'Tampa Bay Rays': 501,
 'San Diego Padres': 294,
 'Houston Astros': 722,
 'Colorado Rockies': 113,
 'New York Yankees': 792,
 'Baltimore Orioles': 2,
 'Chicago White Sox': 401,
 'Atlanta Braves': 556,
 'Oakland Athletics': 446,
 'Los Angeles Dodgers': 802,
 'San Francisco Giants': 53,
 'Cleveland Indians': 441,
 'Chicago Cubs': 425}

In [ ]:
# FULL SEASON

{'Texas Rangers': 166,
 'Los Angeles Angels': 326,
 'Boston Red Sox': 474,
 'Philadelphia Phillies': 344,
 'New York Mets': 356,
 'Kansas City Royals': 11,
 'Minnesota Twins': 575,
 'Pittsburgh Pirates': 55,
 'Milwaukee Brewers': 298,
 'Cincinnati Reds': 338,
 'Miami Marlins': 5,
 'St. Louis Cardinals': 494,
 'Seattle Mariners': 26,
 'Toronto Blue Jays': 110,
 'Washington Nationals': 490,
 'Detroit Tigers': 4,
 'Arizona Diamondbacks': 348,
 'Tampa Bay Rays': 484,
 'San Diego Padres': 322,
 'Houston Astros': 732,
 'Colorado Rockies': 97,
 'New York Yankees': 772,
 'Baltimore Orioles': 0,
 'Chicago White Sox': 418,
 'Atlanta Braves': 535,
 'Oakland Athletics': 466,
 'Los Angeles Dodgers': 798,
 'San Francisco Giants': 70,
 'Cleveland Indians': 436,
 'Chicago Cubs': 450}

In [158]:
print(f"astros win stdev {statistics.stdev(win_totals['Houston Astros'])}")
print(f"nats win stdev {statistics.stdev(win_totals['Washington Nationals'])}")

astros win stdev 4.092108690559616
nats win stdev 4.49299117481547


{'Houston Astros': 50,
 'Oakland Athletics': 50,
 'Philadelphia Phillies': 48,
 'Chicago White Sox': 46,
 'Toronto Blue Jays': 45,
 'Colorado Rockies': 44,
 'Arizona Diamondbacks': 44,
 'Washington Nationals': 44,
 'Los Angeles Dodgers': 43,
 'Cincinnati Reds': 43,
 'New York Mets': 42,
 'Baltimore Orioles': 42,
 'Minnesota Twins': 42,
 'Kansas City Royals': 42,
 'Milwaukee Brewers': 41,
 'Tampa Bay Rays': 41,
 'Seattle Mariners': 40,
 'St. Louis Cardinals': 40,
 'Chicago Cubs': 40,
 'New York Yankees': 40,
 'San Diego Padres': 39,
 'Atlanta Braves': 39,
 'Cleveland Indians': 38,
 'Miami Marlins': 38,
 'San Francisco Giants': 37,
 'Boston Red Sox': 37,
 'Texas Rangers': 37,
 'Los Angeles Angels': 35,
 'Detroit Tigers': 33,
 'Pittsburgh Pirates': 30}